In [1]:
from ipywidgets import Text
from ipyfilechooser import FileChooser
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt
import ipympl
from mpl_toolkits.mplot3d import Axes3D
from lmfit import Parameters, minimize, fit_report
from glob import glob
from os.path import join
import pandas as pd
from correlation_analysis import fit_G_mob_2D

In [2]:
corrchooser = FileChooser(path=r'D:\Data\FFS\210509 da grotein stim'); corrchooser

FileChooser(path='D:\Data\FFS\210509 da grotein stim', filename='', title='HTML(value='', layout=Layout(displa…

In [31]:
xi_range = 20
phi_range = 20

xi, phi  = np.meshgrid(range(-xi_range,xi_range+1),range(-phi_range,phi_range+1))
tau_px   = 16.484848484848484e-6    # pixel dwell time, us
tau_ln   = 4984.090909091016e-6     # line time, us
dr       = 0.05189161484073687      # pixel size, um
w0_g     = 0.177                    # effective beam width for mEGFP, um
w0_y     = 0.177                    # effective beam width for mEYFP, um
w0_r     = 0.196                  # effective beam width for mCherry2, um
w0_gy    = 0.177                   # effective beam width for cross correlations, um
w0_gr    = 0.187                   # effective beam width for cross correlations, um
w0_yr    = 0.187                   # effective beam width for cross correlations, um

n_photon = 1                        # photons for excitation, e.g. n_photon=2 for two-photon excitation
S        = 6                        # ratio of w0 to wz

D_init  = [0, 1, 10]                # diffusion coefficient [lower bound, initial guess, upper bound]
xi0_init = [-20, 8, 20]             # offset in scanning direction [lower bound, initial guess, upper bound]
# xi0_init = 0

xi_snr_min = 20
xi_snr_max = 50
phi_snr_min = 20
phi_snr_max = 50

scanpars = pd.DataFrame(data={'tau_px': [tau_px],
                   'tau_ln': [tau_ln],
                   'dr': [dr],
                   'w0_g': [w0_g],
                   'w0_y': [w0_y],
                   'w0_r': [w0_r],
                   'w0_gy': [w0_gy],
                   'w0_gr': [w0_gr],
                   'w0_yr': [w0_yr],
                   'n_photon': [n_photon],
                   'xi_range': [xi_range],
                   'phi_range': [phi_range],
                   'xi_snr_min': [xi_snr_min],
                   'xi_snr_max': [xi_snr_max],
                   'phi_snr_min': [phi_snr_min],
                   'phi_snr_max': [phi_snr_max]})

In [32]:
corrfiles = glob(join(corrchooser.selected_path, '*.npy'))
labels = [Path(corrfile).stem.split(' corr fxns')[0] for corrfile in corrfiles]
savename = 'fitting output xi0_init=8'

output = pd.DataFrame(index=labels)

try:
    intensities = pd.read_csv(join(corrchooser.selected_path, 'intensities.csv'), index_col=0)
    output = pd.concat([intensities, output], axis=1)
except:
    pass

for corrfile, label in zip(corrfiles, labels):
    corr_fxns = np.load(corrfile)
    nrows, ncols = corr_fxns[0].shape
    xi_center, phi_center = ncols//2, nrows//2
    XI, PHI = np.meshgrid(range(-ncols//2, ncols//2), range(-nrows//2, nrows//2))
    snr_mask = (XI > xi_snr_min) & (XI < xi_snr_max) & (np.abs(PHI) > phi_snr_min) & (np.abs(PHI) < phi_snr_max)
    
    
    G_exp_g = corr_fxns[0][phi_center-phi_range:phi_center+phi_range+1, 
                           xi_center-xi_range:xi_center+xi_range+1]
    G_sigma_g = corr_fxns[1][phi_center-phi_range:phi_center+phi_range+1, 
                           xi_center-xi_range:xi_center+xi_range+1]
    
    G_exp_y = corr_fxns[6][phi_center-phi_range:phi_center+phi_range+1, 
                           xi_center-xi_range:xi_center+xi_range+1]
    G_sigma_y = corr_fxns[7][phi_center-phi_range:phi_center+phi_range+1, 
                           xi_center-xi_range:xi_center+xi_range+1]
    
    G_exp_r = corr_fxns[10][phi_center-phi_range:phi_center+phi_range+1, 
                           xi_center-xi_range:xi_center+xi_range+1]
    G_sigma_r = corr_fxns[11][phi_center-phi_range:phi_center+phi_range+1, 
                           xi_center-xi_range:xi_center+xi_range+1]
    
    G_exp_gy = corr_fxns[2][phi_center-phi_range:phi_center+phi_range+1, 
                           xi_center-xi_range:xi_center+xi_range+1]
    G_sigma_gy = corr_fxns[3][phi_center-phi_range:phi_center+phi_range+1, 
                           xi_center-xi_range:xi_center+xi_range+1]
    
    G_exp_gr = corr_fxns[4][phi_center-phi_range:phi_center+phi_range+1, 
                           xi_center-xi_range:xi_center+xi_range+1]
    G_sigma_gr = corr_fxns[5][phi_center-phi_range:phi_center+phi_range+1, 
                           xi_center-xi_range:xi_center+xi_range+1]
    
    G_exp_yr = corr_fxns[8][phi_center-phi_range:phi_center+phi_range+1, 
                           xi_center-xi_range:xi_center+xi_range+1]
    G_sigma_yr = corr_fxns[9][phi_center-phi_range:phi_center+phi_range+1, 
                           xi_center-xi_range:xi_center+xi_range+1]
    
    
    G0_init_g = [-1, (G_exp_g[phi_range,xi_range-1]+G_exp_g[phi_range,xi_range+1])/2, 10]
    G0_init_y = [-1, (G_exp_y[phi_range,xi_range-1]+G_exp_y[phi_range,xi_range+1])/2, 10]
    G0_init_r = [-1, (G_exp_r[phi_range,xi_range-1]+G_exp_r[phi_range,xi_range+1])/2, 10]
    
    G0_init_gy = [-1, (G_exp_gy[phi_range,xi_range-1]+G_exp_gy[phi_range,xi_range+1])/2, 10]
    G0_init_gr = [-1, (G_exp_gr[phi_range,xi_range-1]+G_exp_gr[phi_range,xi_range+1])/2, 10]
    G0_init_yr = [-1, (G_exp_yr[phi_range,xi_range-1]+G_exp_yr[phi_range,xi_range+1])/2, 10]
    
    if not np.all(np.isnan([G_exp_g, G_exp_y, G_exp_r])):
        fit_g = fit_G_mob_2D(G_exp_g, G_sigma_g, xi, phi, tau_px, tau_ln, dr, 
                        w0_g, S, n_photon, G0_init_g, D_init, xi0_init=0)
        fit_y = fit_G_mob_2D(G_exp_y, G_sigma_y, xi, phi, tau_px, tau_ln, dr, 
                        w0_y, S, n_photon, G0_init_y, D_init, xi0_init=0)
        fit_r = fit_G_mob_2D(G_exp_r, G_sigma_r, xi, phi, tau_px, tau_ln, dr, 
                        w0_r, S, n_photon, G0_init_r, D_init, xi0_init=0)
        
        fit_gy = fit_G_mob_2D(G_exp_gy, G_sigma_gy, xi, phi, tau_px, tau_ln, dr, 
                        w0_gy, S, n_photon, G0_init_gy, D_init, xi0_init)
        fit_gr = fit_G_mob_2D(G_exp_gr, G_sigma_gr, xi, phi, tau_px, tau_ln, dr, 
                        w0_gr, S, n_photon, G0_init_gr, D_init, xi0_init)
        fit_yr = fit_G_mob_2D(G_exp_yr, G_sigma_yr, xi, phi, tau_px, tau_ln, dr, 
                        w0_yr, S, n_photon, G0_init_yr, D_init, xi0_init)
        
        output.loc[label,'G0_init_g'] = fit_g.params['G0'].init_value
        output.loc[label,'G0_fit_g']  = fit_g.params['G0'].value
        output.loc[label,'D_init_g']  = fit_g.params['D'].init_value
        output.loc[label,'D_fit_g']   = fit_g.params['D'].value
        output.loc[label,'xi0_init_g']= fit_g.params['xi0'].init_value
        output.loc[label,'xi0_fit_g'] = fit_g.params['xi0'].value
        output.loc[label,'redchi_g']  = fit_g.redchi
        
        output.loc[label,'G0_init_y'] = fit_y.params['G0'].init_value
        output.loc[label,'G0_fit_y']  = fit_y.params['G0'].value
        output.loc[label,'D_init_y']  = fit_y.params['D'].init_value
        output.loc[label,'D_fit_y']   = fit_y.params['D'].value
        output.loc[label,'xi0_init_y']= fit_y.params['xi0'].init_value
        output.loc[label,'xi0_fit_y'] = fit_y.params['xi0'].value
        output.loc[label,'redchi_y']  = fit_y.redchi
        
        output.loc[label,'G0_init_r'] = fit_r.params['G0'].init_value
        output.loc[label,'G0_fit_r']  = fit_r.params['G0'].value
        output.loc[label,'D_init_r']  = fit_r.params['D'].init_value
        output.loc[label,'D_fit_r']   = fit_r.params['D'].value
        output.loc[label,'xi0_init_r']= fit_r.params['xi0'].init_value
        output.loc[label,'xi0_fit_r'] = fit_r.params['xi0'].value
        output.loc[label,'redchi_r']  = fit_r.redchi
        
        output.loc[label,'G0_init_gy'] = fit_gy.params['G0'].init_value
        output.loc[label,'G0_fit_gy']  = fit_gy.params['G0'].value
        output.loc[label,'D_init_gy']  = fit_gy.params['D'].init_value
        output.loc[label,'D_fit_gy']   = fit_gy.params['D'].value
        output.loc[label,'xi0_init_gy']= fit_gy.params['xi0'].init_value
        output.loc[label,'xi0_fit_gy'] = fit_gy.params['xi0'].value
        output.loc[label,'redchi_gy']  = fit_gy.redchi
        
        output.loc[label,'G0_init_gr'] = fit_gr.params['G0'].init_value
        output.loc[label,'G0_fit_gr']  = fit_gr.params['G0'].value
        output.loc[label,'D_init_gr']  = fit_gr.params['D'].init_value
        output.loc[label,'D_fit_gr']   = fit_gr.params['D'].value
        output.loc[label,'xi0_init_gr']= fit_gr.params['xi0'].init_value
        output.loc[label,'xi0_fit_gr'] = fit_gr.params['xi0'].value
        output.loc[label,'redchi_gr']  = fit_gr.redchi
        
        output.loc[label,'G0_init_yr'] = fit_yr.params['G0'].init_value
        output.loc[label,'G0_fit_yr']  = fit_yr.params['G0'].value
        output.loc[label,'D_init_yr']  = fit_yr.params['D'].init_value
        output.loc[label,'D_fit_yr']   = fit_yr.params['D'].value
        output.loc[label,'xi0_init_yr']= fit_yr.params['xi0'].init_value
        output.loc[label,'xi0_fit_yr'] = fit_yr.params['xi0'].value
        output.loc[label,'redchi_yr']  = fit_yr.redchi

        noise_g  = np.nanstd(corr_fxns[0][snr_mask])
        signal_g = fit_g.params['G0'].value
        snr_g    = signal_g / noise_g
        
        noise_y  = np.nanstd(corr_fxns[6][snr_mask])
        signal_y = fit_y.params['G0'].value
        snr_y    = signal_y / noise_y
        
        noise_r  = np.nanstd(corr_fxns[10][snr_mask])
        signal_r = fit_r.params['G0'].value
        snr_r    = signal_r / noise_r
        
        noise_gy = np.nanstd(corr_fxns[2][snr_mask])
        noise_gr = np.nanstd(corr_fxns[4][snr_mask])
        noise_yr = np.nanstd(corr_fxns[8][snr_mask])
        
        signal_gy = (fit_g.params['G0'].value * fit_y.params['G0'].value)**0.5
        signal_gr = (fit_g.params['G0'].value * fit_r.params['G0'].value)**0.5
        signal_yr = (fit_y.params['G0'].value * fit_r.params['G0'].value)**0.5
        
        snr_gy = signal_gy / noise_gy
        snr_gr = signal_gr / noise_gr
        snr_yr = signal_yr / noise_yr
        
        output.loc[label,'noise_g'] = noise_g
        output.loc[label,'SNR_g']   = snr_g
        output.loc[label,'noise_y'] = noise_y
        output.loc[label,'SNR_y']   = snr_y
        output.loc[label,'noise_r'] = noise_r
        output.loc[label,'SNR_r']   = snr_r
        
        output.loc[label,'noise_gy']  = noise_gy
        output.loc[label,'signal_gy'] = signal_gy
        output.loc[label,'SNR_gy']    = snr_gy
        
        output.loc[label,'noise_gr']  = noise_gr
        output.loc[label,'signal_gr'] = signal_gr
        output.loc[label,'SNR_gr']    = snr_gr
        
        output.loc[label,'noise_yr']  = noise_yr
        output.loc[label,'signal_yr'] = signal_yr
        output.loc[label,'SNR_yr']    = snr_yr
        
output['N_g'] = output['G0_fit_g']**-1
output['N_y'] = output['G0_fit_y']**-1
output['N_r'] = output['G0_fit_r']**-1
output['N_gy'] = output['G0_fit_gy'] * output['N_g'] * output['N_y']
output['N_gr'] = output['G0_fit_gr'] * output['N_g'] * output['N_r']
output['N_yr'] = output['G0_fit_yr'] * output['N_y'] * output['N_r']

output['Q_g'] = output['<I_mEGFP>'] / output['N_g']
output['Q_y'] = output['<I_mEYFP>'] / output['N_y']
output['Q_r'] = output['<I_mCherry2>'] / output['N_r']

output['min(Gg,Gy)'] = np.nanmin(output[['G0_fit_g', 'G0_fit_y']], axis=1)
output['min(Gg,Gr)'] = np.nanmin(output[['G0_fit_g', 'G0_fit_r']], axis=1)
output['min(Gy,Gr)'] = np.nanmin(output[['G0_fit_y', 'G0_fit_r']], axis=1)

output['rel_cc_gy'] = output['G0_fit_gy'] / output['min(Gg,Gy)']
output['rel_cc_gr'] = output['G0_fit_gr'] / output['min(Gg,Gr)']
output['rel_cc_yr'] = output['G0_fit_yr'] / output['min(Gy,Gr)']
    
scanpars.to_csv(join(corrchooser.selected_path, savename+' scanpars.csv'))
output.to_csv(join(corrchooser.selected_path, savename+'.csv'))
output

,<I_mEGFP>,<I_mEYFP>,<I_mCherry2>,# pixels,G0_init_g,G0_fit_g,D_init_g,D_fit_g,xi0_init_g,xi0_fit_g,...,N_yr,Q_g,Q_y,Q_r,"min(Gg,Gy)","min(Gg,Gr)","min(Gy,Gr)",rel_cc_gy,rel_cc_gr,rel_cc_yr
01 post unmixed ROI01,4.677363,0.947657,6.103514,662756.0,0.012413,0.011565,1.0,0.237188,0.0,0.0,...,4.991275,0.054094,0.045168,0.065216,0.011565,0.010685,0.010685,0.364578,0.081021,0.237900
01 pre unmixed ROI01,5.051651,0.993854,6.762745,818884.0,0.013279,0.012178,1.0,0.156024,0.0,0.0,...,6.182776,0.061517,0.043130,0.075537,0.012178,0.011170,0.011170,0.222864,0.107217,0.268311
02 post unmixed ROI01,1.489363,0.348855,2.810850,1654885.0,0.044491,0.044078,1.0,0.272526,0.0,0.0,...,1.199028,0.065649,0.057639,0.056826,0.044078,0.020217,0.020217,0.073303,0.055158,0.198107
02 pre unmixed ROI01,1.623616,0.368399,3.173045,999622.0,0.043635,0.040635,1.0,0.285037,0.0,0.0,...,-0.958506,0.065976,0.050653,0.064649,0.040635,0.020374,0.020374,0.252712,0.037994,-0.131791
03 post unmixed ROI01,0.429615,0.497981,3.752471,569436.0,0.134399,0.150138,1.0,1.988544,0.0,0.0,...,3.297300,0.064502,0.056800,0.066091,0.114061,0.017613,0.017613,-0.060819,-0.146886,0.376095
03 pre unmixed ROI01,0.400337,0.532286,4.104718,431101.0,0.139267,0.127846,1.0,2.295607,0.0,0.0,...,2.264134,0.051181,0.063645,0.064301,0.119569,0.015665,0.015665,0.132888,-0.127369,0.270720
04 post unmixed ROI01,1.372821,1.181123,6.082154,653166.0,0.053691,0.047357,1.0,0.150891,0.0,0.0,...,4.964577,0.065012,0.057100,0.073299,0.047357,0.012051,0.012051,0.031260,-0.047759,0.240007
04 pre unmixed ROI01,1.298830,1.173430,6.470594,768581.0,0.055998,0.050557,1.0,0.122393,0.0,0.0,...,6.196168,0.065665,0.062133,0.071534,0.050557,0.011055,0.011055,0.054480,-0.060742,0.328089
05 post unmixed ROI01,2.218020,1.482216,8.789645,517453.0,0.030392,0.027448,1.0,0.216795,0.0,0.0,...,8.210770,0.060880,0.049994,0.066026,0.027448,0.007512,0.007512,-0.050483,0.113279,0.276942
05 pre unmixed ROI01,2.380975,1.822028,11.089742,906819.0,0.026449,0.025756,1.0,0.239882,0.0,0.0,...,18.062618,0.061324,0.053143,0.078212,0.025756,0.007053,0.007053,0.103673,0.073807,0.526836


In [31]:
output[['D_fit_g', 'D_fit_y', 'D_fit_r',
       'D_fit_gy', 'D_fit_gr', 'D_fit_yr']].astype('float').describe()

,D_fit_g,D_fit_y,D_fit_r,D_fit_gy,D_fit_gr,D_fit_yr
count,20.000000,2.000000e+01,20.000000,2.000000e+01,20.000000,2.000000e+01
mean,0.404647,1.104841e+00,0.415229,4.295075e+00,0.357891,2.924852e+00
std,0.157634,1.657176e+00,0.161369,4.229131e+00,0.177780,4.268051e+00
min,0.065858,6.007639e-11,0.195643,1.290850e-08,0.040881,5.568365e-09
25%,0.315219,3.812359e-01,0.301821,3.936524e-01,0.253547,2.141733e-05
50%,0.403813,5.981216e-01,0.382200,2.764317e+00,0.310031,3.398266e-01
75%,0.488573,1.000000e+00,0.463036,9.861715e+00,0.441334,4.804318e+00
max,0.722064,7.181795e+00,0.828504,9.999998e+00,0.745927,1.000000e+01


In [32]:
output[['rel_cc_gy', 'rel_cc_gr', 'rel_cc_yr']].astype('float').describe()

,rel_cc_gy,rel_cc_gr,rel_cc_yr
count,20.000000,20.000000,20.000000
mean,0.966952,0.677393,2.575340
std,6.001276,0.044591,8.277029
min,-13.265469,0.579145,-2.019880
25%,-0.060507,0.648524,-0.082183
50%,0.084851,0.678178,0.015495
75%,0.206851,0.697760,0.276055
max,19.867996,0.749842,35.023521


In [33]:
output[['xi0_fit_gy', 'xi0_fit_gr', 'xi0_fit_yr']].astype('float').describe()

,xi0_fit_gy,xi0_fit_gr,xi0_fit_yr
count,20.000000,20.000000,20.000000
mean,8.330488,0.796165,11.208519
std,5.492281,0.187930,5.622218
min,-0.229624,0.219349,-1.646180
25%,4.296986,0.806335,7.486301
50%,7.950335,0.862312,10.046024
75%,12.354213,0.888695,15.552847
max,18.112697,0.925796,19.999993


In [34]:
print(
    (output['xi0_fit_gy'] < 3).sum(),
    (output['xi0_fit_gr'] < 3).sum(),
    (output['xi0_fit_yr'] < 3).sum()
)

5 20 1
